In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re


In [2]:

club = "computer"
campus = "Columbus"

d = requests.get(
    #f'https://activities.osu.edu/involvement/student_organizations/find_a_student_org?v=list&s={club}&c={campus}'
    'https://activities.osu.edu/involvement/student_organizations/find_a_student_org?v=list&l=ALL&c=Columbus')
data = d.text
# print(data)


In [18]:
def date_parse(input_str):
    days = ["sundays", "mondays", "tuesdays", "wednesdays", "thursdays", "fridays", "saturdays"]
    regex = "(?:^|\s)((?:(?:M|Mo|T|Tu|Tue|W|We|Wed|Wedn|Wedne|T|Th|Thu|Thur|F|Fr|S|Sa|Sat|Satu|S|Su)|(?:Mon|Tues|Wednes|Thurs|Fri|Satur|Sun)(?:d|da|day)?)s?)\.?,?\s*?(?:@|at)?\s?(\d{3,4}|(?:\d{1,2}:?(?:\d{2})?))\s?(pm|am)?"
    result = re.search(regex, input_str, re.I)

    if result == None:
        parsed_day = 3
        parsed_time = 700
    else:
        day = result.group(1).lower()
        time = result.group(2)
        meridian = result.group(3)
        if meridian:
            meridian = meridian.lower()
        print(day, time, meridian)

        # So, if there is no day list the default will be 4
        # parsed_day = 4
        for i in range(len(days)):
            if days[i].__contains__(day):
                parsed_day = i
                break

        # 7 = 0700
        # 7:20 = 0720
        # 720 = 0720
        # 12 = 1200
        # 7am = 0700
        # 7pm = 1900
        # 7:20am = 0720
        # 7:20pm = 1920

        # Default time is 700
        # parsed_time = 700
        if meridian == None:
            if not time.__contains__(":"):
                if len(time) < 3:
                    parsed_time = int(time) * 100
                else:
                    parsed_time = int(time)
            else:
                time = time.replace(":", "")
                parsed_time = int(time)
        else:   # There is a meridian
            if meridian == "am":
                time = time.replace(":", "")
                if len(time) < 3:
                    parsed_time = int(time) * 100
                else:
                    parsed_time = int(time)
            else:
                time = time.replace(":", "")
                if len(time) < 3:
                    # 700
                    parsed_time = int(time) * 100 + 1200
                else:
                    #UNTESTED
                    parsed_time = int(time) + 1200
            
            if time[0:2] == "12": # and meridian == "am":
                parsed_time -= 1200
            # elif time[0:2] == "12" and meridian == "pm":
            #     parsed_time -= 1200
    
    return parsed_day, parsed_time

In [16]:
# Testing if the date parser works correctly
print(date_parse("Mon 12:50pm")) #Tuesday @ 12:00pm Columbus, OH 43210"))
#print(date_parse("Monday @ 7Columbus, OH 43210"))
print(date_parse("Mondays and fridays 8:30-10:30pm1813 n high st.Columbus, OH 43210"))

mon 12:50 pm
test
(1, 1250)
fridays 8:30 None
test
(5, 830)


In [21]:

soup = BeautifulSoup(data, 'html.parser')

# Good tutorial:
#https://medium.com/geekculture/web-scraping-tables-in-python-using-beautiful-soup-8bbc31c5803e

# Finding the holy table
table = soup.find('table', class_='c-table')
# print(table)
# Collecting Ddata
df = pd.DataFrame(columns=["name", "day", "time", "statement", "topics"])
# df = pd.DataFrame()

# Counter is for testing -- takes a long time to make url requests
counter = 0
for row in table.find_all('tr')[1:]:
    # print(f"Our row is {row}")
    # Find all data for each column
    columns = row.find_all('td')
    club_name = columns[0].text.strip()
    
    #-----------------------------EXTRA DATA--------------------------
    if counter < 9999: #50:
        url = columns[0].find(href=True)['href']
        #url = columns[0].find('a').attrs['href'] <-- alternative code
        #print("Found the URL:", url)

        d2 = requests.get(f"https://activities.osu.edu/{url}")
        current_data = d2.text
        club_site = BeautifulSoup(current_data, 'html.parser')
        #print(club_site)

        current_table = club_site.find('table', class_='c-table')
        #print(current_table)

        # current_columns = current_table.find_all('tr')
        # current_campus = current_columns[0].text.strip()
        # print(current_campus)

        current_counter = 0
        for current_row in current_table.find_all('tr')[1:]:
            property = current_row.th.text.strip()

            # Have to do this because the number of properties is not constant
            if property == "Meeting Time and Place:":
                current_time = current_row.find_all('td')[0].text.strip()
                current_day, current_time = date_parse(current_time)

            # print(property)
            # if current_counter == 14:
            #     current_columns = current_row.find_all('td')
            #     current_campus = current_columns[0].text.strip()
            #     print(current_campus)
            # current_counter += 1
    #-----------------------------------------------------------------

    statement = columns[1].text.strip()
    topics = columns[2].text.strip().split(",")
    topics = [t.strip() for t in topics]
    if topics[-1] == '':
        topics.pop()
    datarow = pd.DataFrame([{"name":club_name.lower(), "day":current_day, "time":current_time, "statement":statement, "topics":set(topics)}])
    df = df.append(datarow, ignore_index=True)
    counter += 1
df

sundays 1700 None
monday 7 pm
wednesdays 7 pm
sunday 6 pm
thursdays 9 None
thursdays 7 pm
thursday 7 None
tuesdays 8 pm
friday 7:00 None
sundays 7 pm
thursdays 6 pm
thursdays 6 None
friday 11:30 am
tuesday 7:05 None
wednesday 11:45 None
w 12 None
thursdays 8:00 None
wednesday 7:30 pm
mondays 6:30 pm
wednesdays 7:00 None
wednesdays 7:00 pm
thursday 7:00 pm
wednesday 7:00 None
wednesday 7 pm
wednesdays 3:30 pm
mondays 6 pm
monday 6:00 None
tuesdays 6 pm
mondays 6:30 pm
wednesdays 7:00 pm
mondays 7:00 pm
monday 7:15 pm
mondays 7 None
saturdays 10 am
mondays 8 pm
monday 4:30 None
monday 4:45 pm
mondays 6:30 pm
sundays 2 pm
mondays 7:30 None
mondays 6:00 pm
mondays 7 pm
tuesdays 7 pm
sunday 7:00 pm
monday 7:30 None
mondays 7:30 pm
monday 6:00 pm
tuesdays 7:00 None
monday 7:30 pm
tuesdays 6:30 pm
monday 6 pm
thursdays 201 None
wednesdays 7:30 pm
w 19 None
w 10 None
thursdays 11:30 None
monday 5 pm
tuesday 6:30 pm
w 19 None
w 12 None
w 19 None
wednesdays 4:30 pm
w 12 None
thursdays 6 pm
wedne

,name,day,time,statement,topics
0,"1 Day For The K.I.A., Inc.",0,1700,1 Day for the K.I.A.’s purpose is to raise awa...,"{Special Interest, Community Service/Service L..."
1,1825 at Ohio State,1,1900,To help students at The Ohio State University ...,"{Ethnic/Cultural, Awareness/Activism, Religiou..."
2,1girl,3,1900,We strive to provide girls and nonbinary stude...,"{Community Service/Service Learning, Awareness..."
3,2022 Intercollegiate Taiwanese American Studen...,3,700,The purpose of this organization is to unite O...,"{Ethnic/Cultural, Columbus}"
4,The 2nd and 7 Foundation Medical Branch,0,1800,The mission of the 2nd and 7 Foundation Medica...,"{Community Service/Service Learning, Awareness..."
...,...,...,...,...,...
1765,"Zeta Phi Beta Sorority, Incorporated",3,700,"To foster the ideas of service, charity, schol...","{Social Fraternities/Sororities, Academic/Coll..."
1766,Zeta Tau Alpha,1,1900,The purpose of Zeta Tau Alpha is the intensify...,"{Social Fraternities/Sororities, Community Ser..."
1767,"Zoo, Wildlife, and Conservation Medicine Club",3,700,"The purpose of the OSU CVM Zoo, Wildlife, and ...","{Special Interest, Academic/College, Columbus}"
1768,Zoology Club at The Ohio State University,3,1830,The purpose of Zoology Club is to enhance the ...,"{Special Interest, Academic/College, Columbus}"


In [22]:
df.to_csv("./club_data.csv")